In [2]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
from multiprocessing import Pool, cpu_count
from itertools import product
import sys
import sqlite3

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v7 import patoms2d
from snapshot_3d_pattern_v6 import patoms3d
from pattern_2d_compare_v4 import pattern_compare_2d
from pattern_3d_compare_v4 import pattern_compare_3d
from get_2d_table_names_v0 import table_names_2d
from updating_ref_table_v0 import update_ref_table
from patom_to_table_v0 import patom_to_table_func

In [3]:
## create in memory 2d database
con2d = sqlite3.connect(":memory:")
cur2d = con2d.cursor()
cur2d.execute('create table pat_2d_000000(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);')
con2d.commit()

In [4]:
## read 2d ref database into memory and convert to ref patoms for comparison
working_ref_patoms = []
with open('/home/gerard/Desktop/capstone_project/ref2d_v2.db', 'rb') as f:
    con2dref = sqlite3.connect(":memory:")
    con2dref.deserialize(f.read())
    cur2dref = con2dref.cursor()
    ref_names = [name for (name,) in cur2dref.execute("select name from sqlite_master where type='table';").fetchall()]
    for i in ref_names:
        table = cur2dref.execute(f"select * from {i};").fetchall()
        # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
        table_array = np.array(table)
        # norm_x = np.array([x[0] for x in table]); norm_y = np.array([x[1] for x in table])
        # pattern_centroid_x = np.array([x[2] for x in table]); pattern_centroid_y = np.array([x[3] for x in table])
        # patom_ind = [x[4] for x in table]; frame_ind = [x[5] for x in table]; patom_time = [x[6] for x in table]

        working_ref_patoms.append(table_array)
ref_patoms_array = np.vstack(working_ref_patoms[:5000])

In [5]:
val = 0
while val == 0:
    rand_array = np.random.random((1, 720, 1280))
    y_len = rand_array.shape[1]
    x_len = rand_array.shape[2]
    # ingest data frame by frame
    for frame in range(rand_array.shape[0]):
        #################################################################################
        ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
        #################################################################################
        frame_patoms = patoms2d(x_len, y_len, rand_array[frame,:,:], frame)
        # patom = [[norm_x, norm_y], [pattern_centroid_x, pattern_centroid_y], patom_ind, frame_ind, patom_time]
        # patom[i][[0][0]: list of x_pos, patom[i][[0][1]: list of y_pos, patom[i][[1][0]: x_cent, patom[i][[1][1]: y_cent, patom[i][[2]: patom_ind, patom[i][[3]: frame_ind, patom[i][[4]: patom_time,
        # num_patoms = len(frame_patoms)
        val += 1
        # ###########################################################################
        # ########## SECOND TASK: COMPARE NEW PATOMS AGAINST REF PATOMS #############
        # ###########################################################################
        # atime = perf_counter()
        # comp_indexes = list(product(range(num_patoms), range(len(working_ref_patoms[:100])))) ## this is the MAJOR limiting factor at the moment
        # with Pool(processes=cpu_count()) as pool:
        #     items = [(frame_patoms[i[0]][0], frame_patoms[i[0]][1], frame_patoms[i[0]][2], frame_patoms[i[0]][3],\
        #             working_ref_patoms[i[1]][0], working_ref_patoms[i[1]][1], working_ref_patoms[i[1]][2], working_ref_patoms[i[1]][3], i) for i in comp_indexes]
        #     ## function outputs ind value of the patom_indexes list, the centroid and distance similarity measures
        #     ########################################################################################################
        #     ########################################################################################################
        #     ###### ATTEMPT TO CONVERT ALL REF TABLES INTO SINGLE NUMPY ARRAY TO COMPAR AGAINST NEW PATTERNS ########
        #     ########################################################################################################
        #     ########################################################################################################
        #     comp_results = pool.starmap(pattern_compare_2d, items)
        #     comp_results = [x for x in comp_results if x is not None]
        #     print("Time to compare 2D patterns with multiprocessing (secs):", (perf_counter()-atime))
        #     # res output: [(i, j)]
        #     # get the remaining new patoms that were not flagged as being similar to previous patterns
        #     remaining_comp_indexes = list(set([x[0] for x in comp_indexes if x not in comp_results]))
        #     ## loop through the output of the comparison function
        #     if comp_results:
        #         for ix, i in enumerate(comp_results):
        #             print('similar')
        #             # get max table number from 2d database
        #             mttime = perf_counter()
        #             next_table_num = int([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()].pop(-1)[-6:]) + 1 
        #             print('time to get max table:',perf_counter()-mttime)
        #             next_table_num = str(next_table_num).zfill(6)
        #             # check if table exists, if so, add patom, if not create table and add patom
        #             ref_table_num = ref_names[i[1]][-6:] # check this is correct
        #             new_patom = frame_patoms[i[0]] # check this is correct
        #             ptttime = perf_counter()
        #             patom_to_table = patom_to_table_func(new_patom)
        #             print('time to convert patom to table:',perf_counter()-ptttime)
        #             # add patom to relevant pattern table get name of ref table to get data table name
        #             if f"pat_2d_{ref_table_num}" in [names for (names,) in cur2d.execute("select name from sqlite_master where type='table';").fetchall()]:
        #                 cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
        #                 con2d.commit()
        #                 # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
        #             else:
        #                 cur2d.execute(f"CREATE TABLE pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
        #                 cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
        #                 con2d.commit()
        #     else:
        #         for i in remaining_comp_indexes:
        #             mttime = perf_counter()
        #             next_table_num = int([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()].pop(-1)[-6:]) + 1 
        #             print('time to get max table:',perf_counter()-mttime)
        #             next_table_num = str(next_table_num).zfill(6)
        #             new_patom = frame_patoms[i] # check this is correct
        #             ptttime = perf_counter()
        #             patom_to_table = patom_to_table_func(new_patom)
        #             print('time to convert patom to table:',perf_counter()-ptttime)
        #             # if patom is not matched then create new pattern & ref tables, get max table name and then create next new one
        #             cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
        #             cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
        #             con2d.commit()


Time to get 2D patterns with multiprocessing (secs): 0.35846385799959535


In [6]:
working_ref_patoms[0].shape

(59, 8)

In [7]:
def cross_join(arr1, arr2):
    # Get the number of rows in each array
    n_rows1 = arr1.shape[0]
    n_rows2 = arr2.shape[0]

    # Repeat each row of arr1 n_rows2 times
    repeated_arr1 = np.repeat(arr1, n_rows2, axis=0)

    # Tile arr2 n_rows1 times
    tiled_arr2 = np.tile(arr2, (n_rows1, 1))

    # Combine the results to form the Cartesian product
    cartesian_product = np.hstack((repeated_arr1, tiled_arr2))

    return cartesian_product

cart_array = cross_join(frame_patoms[0], ref_patoms_array)

In [8]:
x_diff = abs(cart_array[:,0] - cart_array[:,8])
y_diff = abs(cart_array[:,1] - cart_array[:,9])
x_cent_diff = abs(cart_array[:,2] - cart_array[:,10]).min()
y_cent_diff = abs(cart_array[:,3] - cart_array[:,11]).min()

print(x_cent_diff)

1.2997537851333618e-05
